## Meeting Agent

In [7]:
import os

import constants # create your own constants.py file to store api keys
from langchain.chat_models import ChatOpenAI

# OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0.6,
    model_name='gpt-4'
)

/var/folders/s0/r2n9ckp945q8rdtq0rldhjfw0000gn/T/ipykernel_5889/2414119697.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [8]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [13]:
# Perplexity setup
from langchain.tools import BaseTool
import requests
import json
import os

class GetClientDetails(BaseTool):
    name: str = "Client details extractor"
    description: str = "use this tool when you have given a platform name, idea or platform link and you want to find more information about CEO of platform and proposed idea"

    def _run(self, website: str):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "pplx-70b-online",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are artificial intelligence agent which extracts an information by searching about it online"
                        "and returns information in this format if it exists"
                        "Title of website or platform: "
                        "Proposed idea:"
                        "CEO Information:"
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "I am giving you platform name,idea or platform link and you have to find latest information about" 
                        "the platform by going to given link and return atleast 200 words description about idea and"
                        "atleast 150 words description about CEO, their goal and achievements"
                        "Please try to make it as detailed as you can and always refer to online information"
                        "here is the platform link or name "
                        f"{str(website)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {os.environ['PERPLEXITY_API_KEY']}"
        }
        response = requests.post(url, json=payload, headers=headers)
        return response
    
    def _arun(self, website: str):
        raise NotImplementedError("This tool does not support async")


In [15]:
from langchain.tools import BaseTool
import requests
import os

class GetPlatformInfo(BaseTool):
    name: str = "Platform info extractor"
    description: str = "use this tool when you have given a description about a meeting, and you have to find the proposed idea, client name, or platform link from the given description"

    def _run(self, description: str):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "pplx-70b-online",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have been given a meeting description, and you need to find the proposed idea, client name, and website link mentioned in the given description. "
                        "Use client info extractor tool to get more information. "
                        "Return it in this format: "
                        "Idea: <idea> "
                        "Client Info: <client_info> "
                        "Platform Link: <platform_link> "
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "If anything is not provided in the description, use internet and Google search to find the actual information. "
                        "Here is the meeting description: " 
                        f"{str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {os.environ['PERPLEXITY_API_KEY']}"
        }
        response = requests.post(url, json=payload, headers=headers)
        return response

    def _arun(self, website: str):
        raise NotImplementedError("This tool does not support async")


In [18]:
from langchain.tools import BaseTool
import requests
import os

class GetIdeaSolution(BaseTool):
    name: str = "Solution extractor"
    description: str = "use this tool when you have given an idea description and information about client and you have to find solution on how we can achieve the given idea"

    def _run(self, description: str):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "pplx-70b-online",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have been given an idea description and you have to find around 3-4 solutions to achieve the given idea using AI-ML "
                        "or web development technologies. Search online about tech stack, resources, timeline of project, and YouTube videos, and send it with proper formatting and line breaks in this format: "
                        "Solution: <solution> "
                        "Tech stack: <tech_stack> "
                        "Timeline: <timeline> "
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        f"Here is the idea description: {str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {os.environ['PERPLEXITY_API_KEY']}"
        }
        response = requests.post(url, json=payload, headers=headers)
        return response

    def _arun(self, website: str):
        raise NotImplementedError("This tool does not support async")


In [19]:
# Creating Agent
from langchain.agents import initialize_agent
tools = [GetClientDetails(),GetPlatformInfo(),GetIdeaSolution()]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
    handle_parsing_errors=True
)

/var/folders/s0/r2n9ckp945q8rdtq0rldhjfw0000gn/T/ipykernel_5889/1189561063.py:4: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


In [21]:
para = agent(
    "First find the idea, client name and platform link using Platform info extractor tool and find 150 words information about proposed idea about product, goals and achievements of CEO. Once you got the information about their idea then find how we can help them to build it in 200 words"
    "fetch realtime data from internet everytime"
    "This information is going to be added in project proposal so please write in detail and sections like CEO_Info,idea and solution must be explained in 600 words each."
    "Return your response in python dict format like below and please use proper line breaks: "
    """
    "platform_name": platform_name
    "CEO_Name": ceo_name
    "idea": idea
    "solution": solution
    "tech_stack": tech stack
    "timeline": timeline
    "Platform_link": platform_link
    """
    "Here is the description:"
    "Hey there, myself rohan and we are hosting this meeting to discuss about my idea of making a twitter automation tool with several features like automated tweets, scheduled tweets, tweet improvement guides etc"
)

# Print the raw response to see what's returned
print("Agent Response:", para)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I need to extract the proposed idea, client name, and platform link from the given meeting description.
Action: Platform info extractor
Action Input: Hey there, myself rohan and we are hosting this meeting to discuss about my idea of making a twitter automation tool with several features like automated tweets, scheduled tweets, tweet improvement guides etc
Observation: <Response [400]>
Thought:The Platform info extractor tool didn't return any results. It seems like there's no platform link included in the description. However, the client's name "Rohan" and the idea of making a "Twitter automation tool" were mentioned.
Action: Client details extractor
Action Input: Rohan, Twitter automation tool
Observation: <Response [400]>
Thought:The Client details extractor tool also didn't return any results. It seems the tool couldn't find more information about the CEO and proposed idea based on the given input.
Action: Platform info extractor
Action Input: Twitter automation tool
Observation: <

In [25]:
from openai import OpenAI
import os
import json

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"""
                I am giving you some data in python dict format and you will have to add more information by searching about it online in given data so that it can be added in project proposal but don't add any fake information. Once you got the information then return the data in the same format.
                Try to add more information by yourself too to make it more detailed and make it around 1000 words. Don't return anything except this dictionary and also please use proper line breaks in every paragraph of each section of dict.
                Here is the information about fields in the given dict: 
                platform_name: Name of platform
                CEO_Info: Background about CEO and information about CEO
                CEO_Name: Name of CEO
                idea: project idea
                solution: solution on how we can solve the given problem statement and how to achieve the given idea
                tech_stack: tech stack used to build the solution
                timeline: timeline for the project
                Platform_link: link to the platform
                Here is the data: 
                {para["output"]}
            """,
        }
    ],
    model="gpt-4",
    temperature=0.7
)

# Print the raw output
print("Raw GPT-4 Response:", chat_completion.choices[0].message.content)

# Now let's safely try to parse the output as JSON
try:
    output = json.loads(chat_completion.choices[0].message.content)
    print("Parsed Output:", output)
except json.JSONDecodeError:
    print("Error: The response is not valid JSON. Here's the raw output:")
    print(chat_completion.choices[0].message.content)


Raw GPT-4 Response: Here is the updated information in the Python dictionary format:

```python
project_info = {
    'platform_name': 'Twitter Automation Tool',
    'CEO_Info': 'Rohan is a proficient software engineer with over 10 years of experience in the field. He has a bachelor’s degree in Computer Science from the Indian Institute of Technology (IIT) and a master’s degree in Computer Science from Stanford University. Before founding this platform, he worked with major tech giants like Google and Microsoft. His area of expertise lies in automation and machine learning. He has a deep understanding of social media platforms and their APIs which led him to the creation of this Twitter Automation Tool.',
    'CEO_Name': 'Rohan',
    'idea': 'The idea is to create a Twitter automation tool. The tool will automate several tasks such as scheduling tweets, following and unfollowing users, retweeting, replying to tweets, and so on. This will help businesses and individuals to manage their T

In [27]:
# Code for creating a markdown for our proposal
from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"""
                I am giving you some data in python dict format and you will have to add more information in it and convert it in markdown file with below format
                Don't use anything except headings and paragraphs in markdown
                # Project name
                project name
                # Who is the client?
                information about client
                # What is the idea?
                information about idea
                # How can we help?
                detailed information about solution
                # Tech stack
                information about tech stack
                # Timeline
                information about timeline
                Try to add more information by yourself too to make it more detailed and make it around 800 words. Don't return anything except markdown and use proper line breaks.
                Here is the information about fields in the given dict: 
                platform_name: Name of platform
                CEO_Info: Background about CEO and information about CEO
                CEO_Name: Name of CEO
                idea: project idea
                solution: solution on how we can solve the given problem statement and how to achieve the given idea
                tech_stack: tech stack used to build the solution
                timeline: timeline for the project
                Platform_link: link to the platform
                Here is the data: 
                {para["output"]}
            """,
        }
    ],
    model="gpt-4",
    temperature=0.7
)
output = chat_completion.choices[0].message.content
print(output)
# output = json.loads(output)
# print(json_output["output"])

# Project Name
Twitter Automation Tool

# Who is the client?
The client for this project is Rohan. Although we don't have much background information about Rohan, what we know is that he is an individual who is seeking to streamline his Twitter usage. He is forward-thinking, appreciates the power of automation, and understands the potential of leveraging technology to enhance productivity and efficiency.

# What is the idea?
Rohan's idea is to create a Twitter Automation Tool. This tool aims to automate certain repetitive tasks on Twitter to save time and improve efficiency. With the surge in the usage of social media platforms like Twitter, manually managing all the activities can be quite a daunting task. Rohan, understanding the potential of automation in this sphere, has proposed this idea. The tool could potentially automate tasks such as tweeting, retweeting, liking tweets, following/unfollowing users, and much more.

# How can we help?
To bring Rohan's idea to life, we would nee

In [31]:
!pip install pypandoc

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [32]:
import pypandoc
output = pypandoc.convert_text(output, 'docx', format='md', outputfile="output.docx")
# output = json.loads(output.output)
print(output)

OSError: No pandoc was found: either install pandoc and add it
to your PATH or or call pypandoc.download_pandoc(...) or
install pypandoc wheels with included pandoc.

In [35]:
para = agent(
    """
    First, gather information about the company Emerson Electric and their idea about implementing an AI-driven manufacturing process to enhance operational efficiency. 
    Then, provide solutions on how we can help them achieve this goal, including technologies and strategies that can be used.
    Please generate a professional project proposal including the following sections: 
    1. Client Information (Emerson Electric) 
    2. Idea (AI Manufacturing Process) 
    3. Potential Solutions for Enhancing Operational Efficiency 
    4. Tech Stack 
    5. Timeline
    Please write this information in detail and return it in Python dictionary format.
    """
)

output = json.loads(para["output"])
print(output)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I need to gather information about the company Emerson Electric and their proposed idea, then I need to find a solution on how we can help them achieve this goal.
Action: Client details extractor
Action Input: Emerson Electric
Observation: <Response [400]>
Thought:The client details extractor did not work. I need to try a different approach to gather information about Emerson Electric and their proposed idea.
Action: Platform info extractor
Action Input: Emerson Electric AI Manufacturing Process
Observation: <Response [400]>
Thought:The platform info extractor also did not work. I may need to manually research the information about Emerson Electric and their AI Manufacturing Process proposal.
Action: Manual research
Action Input: Emerson Electric, AI Manufacturing Process
Observation: Manual research is not a valid tool, try one of [Client details extractor, Platform info extractor, Solution extractor].
Thought:Final Answer: I'm sorry, the provided tools don't seem to be able to extrac

JSONDecodeError: Expecting value: line 1 column 1 (char 0)